In [4]:
import pandas as pd
import numpy as np

data = pd.read_csv('train.csv')
Y = data.loc[:,'purchase']
X = data.iloc[:, :-1]
X.set_index('id')
X.head(5)

,id,num_of_admin_pages,admin_page_duration,num_of_info_pages,info_page_duration,num_of_product_pages,product_page_duration,total_duration,BounceRates,ExitRates,...,Month,device,internet_browser,Region,user_type,Weekend,A,B,C,D
0,0,0.0,0.0,0.0,0.0 minutes,1.0,0.0 minutes,NaN,0.200000,0.200000,...,Feb,1.0,safari_15,1.0,Returning_Visitor,False,c_1,118.880094,log202,NaN
1,1,0.0,0.0,0.0,0.0 minutes,1.0,0.0 minutes,0.000000,0.200000,0.200000,...,Feb,4.0,safari_14,9.0,Returning_Visitor,False,c_3,113.358423,log404,NaN
2,2,0.0,0.0,0.0,NaN,NaN,627.5 minutes,627.500000,0.020000,0.050000,...,Feb,3.0,browser_3_v17,1.0,Returning_Visitor,True,c_4,121.507695,log202,NaN
3,3,0.0,0.0,0.0,0.0 minutes,19.0,154.2166667 minutes,154.216667,0.015789,0.024561,...,Feb,2.0,chrome_99.1.3,1.0,Returning_Visitor,False,c_3,93.747176,log_100,NaN
4,4,0.0,0.0,0.0,0.0 minutes,1.0,0.0 minutes,NaN,0.200000,0.200000,...,Feb,2.0,edge_96.0.1054.75,3.0,Returning_Visitor,False,c_3,99.545824,log202,NaN


In [12]:
Y.iloc[X.loc[~np.isnan(X.D)].id]

34       0
179      0
222      0
328      0
346      0
        ..
9779     0
9924     0
10282    0
10388    0
10430    0
Name: purchase, Length: 105, dtype: int64

In [13]:
Y

0        0
1        0
2        0
3        0
4        0
        ..
10474    0
10475    0
10476    0
10477    0
10478    0
Name: purchase, Length: 10479, dtype: int64